<a href="https://colab.research.google.com/github/Omar61554/FAI_autonomous-car/blob/master/udacity_car_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the necessary libraries and mounting Google Drive (if needed)

In [207]:
import numpy as np
import pandas as pd
import math
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import os
import csv
import random
# Mount Google Drive (if needed)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Define the CNN model architecture

In [208]:

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 24, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(24, 36, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(36, 48, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(48, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1)
        )
        self.fc1 = nn.Linear(64 * 6 * 6, 96)
        self.fc2 = nn.Linear(96, 100)

        # Linear regression branch
        self.linear_regression = nn.Linear(100, 96)

        self.fc3 = nn.Linear(100, 10)
        self.fc4 = nn.Linear(10, 1)


    def forward(self, x):
        batch_size, seq_len, channels, height, width = x.size()

        # Process input through convolutional layers
        x = x.view(batch_size * seq_len, channels, height, width)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)

        # Calculate the appropriate padding manually
        padding_height = (x.size(-2) - 1) // 2
        padding_width = (x.size(-1) - 1) // 2
        self.linear_regression = nn.Conv2d(64, 1024, kernel_size=1, stride=1, padding=0)

        # Continue with the original path
        x = self.fc3(x)
        x = self.fc4(x)

        # Reshape back to the original sequence length
        x = x.view(batch_size, seq_len, -1)

        return x, x

Define the custom dataset class

In [209]:
class CustomDataset(Dataset):
    def __init__(self, data_path, transform=None):
        self.data = []
        with open(data_path, 'r') as file:
            reader = csv.reader(file)
            for line in reader:
                image_paths = line[:3]  # Extract the first three values as image paths
                label = float(line[6])  # Extract the last value as the label
                self.data.append({'image_path': image_paths, 'label': label})
        self.transform = transform
    def __getitem__(self, index):
      image_paths = self.data[index]['image_path']
      images = []

      for image_path in image_paths:
          image = cv2.imread(os.path.join(data_path, image_path))
          if image is not None:
              images.append(image)
          else:

              images.append(np.zeros((160, 320, 3), dtype=np.uint8))

      angle = torch.tensor(self.data[index]['label'], dtype=torch.float32)

     # Apply transformations if specified
      if self.transform:
        images = [self.transform(image) for image in images]
        if len(images) < 3:

            images.extend([torch.zeros_like(images[0])] * (3 - len(images)))
        images = torch.stack(images).float()  # Convert the list of tensors to a single tensor
         # Reshape and expand dimensions of the angle tensor
        angle = angle.view(-1, 1, 1).expand(-1, 3, 1)
        return images, angle

    def __len__(self):
        return len(self.data)

 Load and preprocess the data

In [211]:
# Define the paths to data
data_path = '/content/drive/MyDrive/sim data/data/driving_log.csv'


# Define the ratio for splitting the data into test and validation sets
test_ratio = 0.2  # 20% of the data will be used for testing

# Create the dataset
transform = ToTensor()  # Add any other transformations if needed
dataset = CustomDataset(data_path, transform=transform)

# Calculate the number of samples for the test and validation sets
num_samples = len(dataset)
num_test_samples = int(test_ratio * num_samples)
num_val_samples = num_samples - num_test_samples

# Split the dataset into test and validation sets
test_dataset, val_dataset = torch.utils.data.random_split(dataset, [num_test_samples, num_val_samples])

# Create the data loaders
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

Initialize the model, define the loss function and optimizer

In [212]:
# Initialize the model
model = CNNModel()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


Training the model

In [213]:
# Training
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


for epoch in range(num_epochs):
    running_loss = 0.0

    for images, angles in train_loader:
        images = images.to(device)
        angles = angles.to(device)

        optimizer.zero_grad()
        outputs, mapped_outputs = model(images)

        loss = criterion(outputs, angles)
        mapped_loss = criterion(mapped_outputs, angles)
        total_loss = loss + mapped_loss

        total_loss.backward()
        optimizer.step()

        running_loss += total_loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1} - Training Loss: {epoch_loss}")


Epoch 1 - Training Loss: 359.4897592166528
Epoch 2 - Training Loss: 11.588416777105435
Epoch 3 - Training Loss: 10.101915077562817
Epoch 4 - Training Loss: 10.93315867124434
Epoch 5 - Training Loss: 10.0560758115654
Epoch 6 - Training Loss: 11.36872775524663
Epoch 7 - Training Loss: 12.075332904832168
Epoch 8 - Training Loss: 12.188220989855534
Epoch 9 - Training Loss: 11.312747571278702
Epoch 10 - Training Loss: 11.016983746386627


Evaluation on validation set

In [216]:
model.eval()
val_loss = 0.0

with torch.no_grad():
    for images, angles in val_loader:
        images = images.to(device)
        angles = angles.to(device)
        # Reshape the target tensor to match the input size
        angles = angles.view(angles.size(0), angles.size(2), angles.size(3))
        outputs, _ = model(images)
        loss = criterion(outputs, angles)
        val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss}")

Validation Loss: 3.8459620624780655


Evaluation on the test set

In [217]:
# Evaluation on test set
model.eval()
test_loss = 0.0
with torch.no_grad():
    for images, angles in test_loader:
        images = images.to(device)
        angles = angles.to(device)
        # Reshape the target tensor to match the input size
        angles = angles.view(angles.size(0), angles.size(2), angles.size(3))
        outputs, _ = model(images)
        loss = criterion(outputs, angles)
        test_loss += loss.item()

print(f"Test Loss: {test_loss / len(test_loader)}")

Test Loss: 7.86317600607872


Saving the trained model

In [218]:
# Save the trained model
torch.save(model.state_dict(), 'model.pthT1')